### Lesson_3 
1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.
2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты).

1 добавил связанные с бд фичи, поменял просто код на функцию <br>
2 изменил функцию чтобы она была как запись в бд, а не в список<br>
3 старые данные не потёр, но спрятал за #

In [1]:
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient
from pprint import pprint
from pymongo.errors import DuplicateKeyError as dke
import pandas as pd
import numpy as np

In [2]:
def comp(string):
    data = {}
    x = string.split()
    if len(x) == 0:
        data['min'] = 'NaN'
        data['max'] = 'NaN'
        data['cur'] = 'NaN'
        return data
    if x[0] == 'от':
        data['min'] = int(x[1])
        data['max'] = 'NaN'
        data['cur'] = x[-1]
        return data
    if x[0] == 'до':
        data['min'] = 'NaN'
        data['max'] = int(x[1])
        data['cur'] = x[-1]
        return data
    try:
        int(x[0])
        data['min'] = int(x[0])
        data['max'] = int(x[2])
        data['cur'] = x[-1]
        return data
    except ValueError:
        print('ошибка при обработке списка ',x)
        print('??????????')

In [3]:
def search_vacancy(dom): 
    vacancy_list = []
    count = 0
    for i in range(int(dom.find_all('a', {'class': 'bloko-button'})[-2].text)):
        response = requests.get(f'{base_url_hh}{search_hh}{search_input_hh}{page}{i}', headers=headers)
        dom = BeautifulSoup(response.text, 'html.parser')
        vacancy_data = dom.find_all('div', {'class':'vacancy-serp-item'})
        for el in vacancy_data:
            temp_data ={}
            vacancy_title = el.find('a', {'data-qa': 'vacancy-serp__vacancy-title'}).text
            try:
                vacancy_comp = el.find('span', {'data-qa':"vacancy-serp__vacancy-compensation"}).text.replace('\u202f','')
            except AttributeError:
                vacancy_comp = ''
            vacancy_href = el.find('a', {'data-qa': 'vacancy-serp__vacancy-title'})['href']
            vacancy_id = vacancy_href
            vacancy_id = vacancy_id.replace("https://saratov.hh.ru/vacancy/", " ")
            vacancy_id = vacancy_id.replace('?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=Python%20junior', ' ')
            vacancy_id = int(vacancy_id)
    #         print(vacancy_id)
            temp_data['_id'] = vacancy_id
            temp_data['title'] = vacancy_title
            temp_data['min_salary'] = comp(vacancy_comp)['min']
            temp_data['max_salary'] = comp(vacancy_comp)['max']        
            temp_data['currency'] = comp(vacancy_comp)['cur']
            temp_data['href'] = vacancy_href
            if vacancy_href.find('hh'):
                temp_data['site'] = 'HeadHunter'
            else: 
                temp_data['site'] = 'Superjob'
#             vacancy_list.append(temp_data)
            try:
                vacancy.insert_one(temp_data)
                count += 1
            except dke:
                pass
    print(f'найдено {count} новых вакансий' )

In [4]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36'}
sity = 'saratov'
base_url_hh = f'https://{sity}.hh.ru/'
search_hh = 'search/vacancy?area=&fromSearchLine=true&text='
search_input_hh = 'Python+junior'
page = '&page='

# https://www.superjob.ru/vacancy/search/?keywords=python%20junior&page=1
# base_url_sj = 'https://www.superjob.ru/'
# search_sj = 'vacancy/search/?keywords='
# search_input_sj = 'python'
#search/vacancy?area=&fromSearchLine=true&text=Python+junior&page=0&hhtmFrom=vacancy_search_list

In [5]:
response = requests.get(base_url_hh+search_hh+search_input_hh, headers=headers)

In [6]:
dom = BeautifulSoup(response.text, 'html.parser')

In [8]:
client = MongoClient('localhost', 27017)
db = client['vacancies']
vacancy = db.vacancy

In [10]:
# vacancy_list, len(vacancy_list)

In [11]:
# response = requests.get(base_url_sj+search_sj+search_input_sj+page+'1', headers=headers)
# dom = BeautifulSoup(response.text, 'html.parser')
# print(base_url_sj+search_sj+search_input_sj+page+'1')
# vacancy_data = dom.find_all('div', {'class':'vacancy-serp-item'})
# n = int(dom.find_all('a', {'class': 'bloko-button'})[-2].text)
# n
# vacancy_data[1].find('a', {'data-qa': 'vacancy-serp__vacancy-title'})
# vacancy_data[1].find('span', {'data-qa':"vacancy-serp__vacancy-compensation"}).text.replace('\u202f','')
# vacancy_data[1].find('a', {'data-qa': 'vacancy-serp__vacancy-title'})['href']
# f-test-search-result-item


In [12]:
# pprint(dom.find_all('div', {'class': '_3zucV _3bZkM _1hWf- _1PXgQ _3bO-Z l9LnJ'}))

In [13]:
# vacancy_df = pd.DataFrame(vacancy_list)
# vacancy_df

In [14]:
# vacancy_df = vacancy_df.replace('NaN', np.nan)
# vacancy_df.dtypes

In [15]:
# vacancy_df.to_csv('vacancy.csv')

In [16]:
# try:
#     vacancy.insert_one({"_id": ,# id = id from site or generate id from link,
#                                 # но сгенерированный id всё равно будет такой же как на сайте, просто замудрёный
        
#     })
# ну в общем нужно просто изменить изначальную функцию для отправки в бд, а не в создание vacancy_list

In [17]:
search_vacancy(dom)

найдено 6 новых вакансий


In [18]:
count_1 = 0
for doc in vacancy.find({}):
    count_1 +=1
print(count_1) # 497 вакансий до поиска новых


497


In [20]:
search_vacancy(dom)

найдено 5 новых вакансий


In [21]:
count_1 = 0
for doc in vacancy.find({}):
    count_1 +=1
print(count_1) # 502 вакансий после поиска новых

502


конечно мне суют новые вакансии, которые даже близко не подходят под мой запрос.<br>
отдельно джун, отдельно python и ещё сверху любых профессий хоть как-то связанных с it

In [22]:
for doc in vacancy.find({}):
    pprint(doc)

{'_id': 52061597,
 'currency': 'NaN',
 'href': 'https://saratov.hh.ru/vacancy/52061597?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=Python%20junior',
 'max_salary': 'NaN',
 'min_salary': 'NaN',
 'site': 'HeadHunter',
 'title': 'Junior Data Engineer (Intern)'}
{'_id': 52203762,
 'currency': 'руб.',
 'href': 'https://saratov.hh.ru/vacancy/52203762?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=Python%20junior',
 'max_salary': 80000,
 'min_salary': 40000,
 'site': 'HeadHunter',
 'title': 'Junior разработчик Python Django'}
{'_id': 53184327,
 'currency': 'руб.',
 'href': 'https://saratov.hh.ru/vacancy/53184327?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=Python%20junior',
 'max_salary': 190000,
 'min_salary': 150000,
 'site': 'HeadHunter',
 'title': 'Junior Data Engineer (Python)'}
{'_id': 53237122,
 'currency': 'руб.',
 'href': 'https://saratov.hh.ru/vacancy/53237122?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=Python%20junior',
 '

 'currency': 'NaN',
 'href': 'https://saratov.hh.ru/vacancy/49714165?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=Python%20junior',
 'max_salary': 'NaN',
 'min_salary': 'NaN',
 'site': 'HeadHunter',
 'title': 'SDET (QA Automation Engineer)/Trainee'}
{'_id': 51334447,
 'currency': 'NaN',
 'href': 'https://saratov.hh.ru/vacancy/51334447?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=Python%20junior',
 'max_salary': 'NaN',
 'min_salary': 'NaN',
 'site': 'HeadHunter',
 'title': 'Преподаватель по Data Science (Machine Learning)'}
{'_id': 48378400,
 'currency': 'NaN',
 'href': 'https://saratov.hh.ru/vacancy/48378400?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=Python%20junior',
 'max_salary': 'NaN',
 'min_salary': 'NaN',
 'site': 'HeadHunter',
 'title': 'Ведущий Java-разработчик'}
{'_id': 44049175,
 'currency': 'NaN',
 'href': 'https://saratov.hh.ru/vacancy/44049175?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=Python%20junior',
 'max_